The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l


# 编译器和解释器
:label:`sec_hybridize`

到目前为止，这本书主要关注命令式编程，它使用诸如`print`、`+`和`if`之类的语句来更改程序的状态。考虑下面这个简单的命令式程序的例子。


In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


Python是一种*解释语言*（interpreted language）。当评估上述 `fancy_func` 函数时，它按顺序执行函数体的操作。也就是说，它将计算`e = add(a, b)`，并将结果存储为变量`e`，从而更改程序的状态。接下来的两个语句`f = add(c, d)`和`g = add(e, f)`将类似地执行，执行加法并将结果存储为变量。 :numref:`fig_compute_graph` 说明了数据流。

![命令式编程中的数据流。](http://d2l.ai/_images/computegraph.svg)
:label:`fig_compute_graph`

尽管命令式编程很方便，但可能效率低下。一方面，即使 `add` 函数在 `fancy_func` 中被重复调用，Python也会单独执行这三个函数调用。如果在一个GPU（甚至多个GPU）上执行这些命令，那么Python解释器产生的开销可能会非常大。此外，它需要保存`e`和`f`的变量值，直到 `fancy_func` 中的所有语句都执行完毕。这是因为我们不知道在执行语句 `e = add(a, b)` 和 `f = add(c, d)` 之后，程序的其他部分是否会使用变量`e`和`f`。

## 符号式编程

考虑另一种选择*符号式编程*（symbolic programming），通常只在完全定义了过程之后才执行计算。这个策略被多个深度学习框架使用，包括Theano和TensorFlow（后者已经获得了命令式编程扩展）。通常包括以下步骤：

1. 定义要执行的操作。
1. 将操作编译成可执行程序。
1. 提供所需的输入并调用编译后的程序供执行。

这允许进行大量优化。首先，在许多情况下，我们可以跳过Python解释器。从而消除在多个更快的GPU上与在CPU上的单个Python线程搭配使用时可能出现的性能瓶颈。其次，编译器可能会优化并将上述代码重写为`print((1 + 2) + (3 + 4))`甚至`print(10)`。这是可能的，因为编译器在将其转换为机器指令之前可以看到完整的代码。例如，只要不再需要某个变量，它就可以释放内存（或者从不分配内存）。或者它可以将代码转换为一个完全等价的片段。为了获得更好的想法，请考虑下面的命令式编程的模拟。


In [2]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


命令式（解释式）编程和符号式编程的区别如下：

* 命令式编程更容易。在Python中使用命令式编程时，大多数代码都是简单易懂的。调试命令式编程代码也更容易。这是因为更容易获取和打印所有相关的中间变量值，或者使用Python的内置调试工具。
* 符号式编程更高效，更易于移植。符号式编程使得在编译期间优化代码更加容易，同时还能够将程序移植到独立于Python的格式中。这允许程序在非Python环境中运行，从而避免了与Python解释器相关的任何潜在性能问题。

## 混合式编程

历史上，大多数深度学习框架在命令式方法和符号式方法之间进行选择。例如，Theano、TensorFlow（灵感来自前者）、Keras和CNTK采用了符号式方法。相反地，Chainer和PyTorch采取了命令式方法。在后来的更新版中，tensorflow2.0和Keras增加了命令式方法。


如上所述，PyTorch基于命令式编程并使用动态计算图。为了利用符号式编程的可移植性和效率，开发人员考虑了是否有可能将两种编程模型的优点结合起来。这就产生了torchscript，它允许用户使用纯命令式编程进行开发和调试，同时能够将大多数程序转换为符号式程序，以便在需要产品级计算性能和部署时运行。


## `Sequential`的混合式编程

要了解混合式编程的工作原理，最简单的方法是考虑具有多层的深层网络。按照惯例，Python解释器将需要为所有层执行代码以生成指令，然后可以将该指令转发到CPU或GPU。对于单个（快速）计算设备，这不会导致任何重大问题。另一方面，如果我们使用高级的8-GPU服务器，比如AWS P3dn.24xlarge实例，Python将很难让所有GPU保持忙碌。单线程Python解释器成为这里的瓶颈。让我们看看如何通过用`HybridSequential`替换代码重要部分的`Sequential`来解决问题。我们首先定义一个简单的多层感知机。


In [3]:
import torch
from torch import nn
from d2l import torch as d2l

# 生产网络的工厂
def get_net():
    net = nn.Sequential(nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 128),
                        nn.ReLU(), nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.0274,  0.0237]], grad_fn=<AddmmBackward>)

通过使用`torch.jit.script`函数转换模型，我们可以编译和优化多层感知机的计算。模型的计算结果保持不变。


In [4]:
net = torch.jit.script(net)
net(x)

tensor([[-0.0274,  0.0237]], grad_fn=<AddmmBackward>)

编写与以前相同的代码，并使用`torch.jit.script`简单地转换模型。一旦发生这种情况，网络将得到优化（我们将在下面对性能进行基准测试）。


### 通过混合式编程加速

为了证明通过编译获得的性能改进，我们比较了混合编程前后执行`net(x)`所需的时间。让我们先定义一个函数来度量这个时间。当我们开始衡量（和改进）性能时，它在本章中将非常有用。


In [5]:
#@save
class Benchmark:
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

现在我们可以调用网络两次，一次使用torchscript，一次不使用torchscript。


In [6]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000):
        net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'):
    for i in range(1000):
        net(x)

无torchscript: 1.9191 sec


有torchscript: 2.2802 sec


从上面的结果中可以看到，使用`nn.Sequential`函数编写了`torch.jit.script`实例的脚本之后，通过使用符号式编程来提高计算性能。


### 序列化


编译模型的好处之一是我们可以将模型及其参数序列化（保存）到磁盘。这允许我们以独立于所选前端语言的方式存储模型。这使我们能够将经过训练的模型部署到其他设备上，并方便地使用其他前端编程语言。同时，代码通常比命令式编程更快。让我们看看`save`的实际功能。


In [7]:
net.save('my_mlp')
!ls -lh my_mlp*

-rw-r--r-- 1 jenkins jenkins 651K May 16 01:50 my_mlp


## 小结

* 命令式编程使设计新模型变得容易，因为它可以用控制流编写代码，并且能够使用大量Python软件生态。
* 符号式编程要求我们在执行程序之前指定并编译程序。其好处是提高了性能。


## 练习


1. 回顾前几章中你感兴趣的模型。你能通过重新实现它们来提高它们的计算性能吗？


[Discussions](https://discuss.d2l.ai/t/2788)
